In [20]:
from qiskit_aer.primitives import Estimator           # V1 primitive
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Pauli, SparsePauliOp
import numpy as np, time

def heisenberg_op(n, jx, jy, jz=0.):
    paulis, coeffs = [], []
    for i in range(n-1):
        pad = lambda p: "I"*i + p + p + "I"*(n-i-2)
        paulis += [Pauli(pad("X")), Pauli(pad("Y")), Pauli(pad("Z"))]
        coeffs += [jx, jy, jz]
    return SparsePauliOp(paulis, coeffs=np.array(coeffs, complex))

n = int(input("Number of qubits (sites): "))
estimator = Estimator(skip_transpilation=True)        # ← fix
ansatz    = TwoLocal(n, ['ry', 'rz'], 'cx', entanglement='linear', reps=2)
vqe       = VQE(estimator, ansatz, SLSQP(maxiter=1000))

rng = np.random.default_rng()
J1, J2 = rng.uniform(0.5,2.0,1000), rng.uniform(0.5,2.0,1000)
t0 = time.perf_counter()
for jx, jy in zip(J1, J2):
    e0 = vqe.compute_minimum_eigenvalue(heisenberg_op(n, jx, jy)).eigenvalue.real
print(f"Runtime: {time.perf_counter()-t0:.2f} s")


Number of qubits (sites):  3


AlgorithmError: 'The primitive job to evaluate the energy failed!'